In [1]:
from pandas import read_csv
from ticktack import load_presaved_model
from ticktack.fitting import SingleFitter
from operator import methodcaller
from jax.numpy.linalg import pinv
from jax.numpy import where, array, arange, full, exp, vstack
from jax.numpy import float32
from plotnine import *

In [2]:
def super_gaussian(time, start_time):
    DURATION = 0.7751569    # From `get_template`
    AREA = 5.59385463       # From `get_template`

    middle = start_time + DURATION / 2 # Middle of event 
    height = AREA / DURATION    # Amplitude of event
    gauss = height * exp(- ((time - middle) / (DURATION / 2)) ** 16)   # Gaussian pulse

    return gauss

In [3]:
#! Can possibly get all of this from the fitting object as `jax` types instantly
injection_data = read_csv("IR_Data.csv")    # Loading the entire dataset
injection_data = injection_data[injection_data["year"] < 825]   # Subsetting into test section
injection_data = injection_data[injection_data["year"] > 725]   # Lower bound 

In [4]:
carbon_box_model = load_presaved_model("Guttler14", 
    production_rate_units="atoms/cm^2/s")   # Creating a test model 
fitting_object = SingleFitter(carbon_box_model) # Test fitting object
fitting_object.load_data("IR_Data.csv") #! Reloading the data
fitting_object.prepare_function(model=super_gaussian)   
dc14 = getattr(fitting_object, "dc14")

INFO[2022-01-17 08:35:18,344]: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO[2022-01-17 08:35:18,345]: Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO[2022-01-17 08:35:18,346]: Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
WARNING[2022-01-17 08:35:18,347]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [6]:
# (ggplot(injection_data, aes(x="year", y="d14c",))
#     + theme_bw()
#     + geom_point(color=where(injection_data["id"] == "Intcal20", "black", "red")))


In [ ]:
for year in range(750, 801):
    chunked_data = injection_data[injection_data["year"] > year - 25]
    chunked_data = chunked_data[chunked_data["year"] < year + 25]
    
    chunked_years = chunked_data["year"]
    fitting_object.time_data = array([*chunked_years], dtype=float32)
    chunked_event = dc14(array([year], dtype=float32))
    chunked_const = full(chunked_years.shape, 1.0)
    
    design_matrix = vstack([chunked_const, chunked_years, chunked_event])
    

Pick a 100 year block and test the code. So I could fill in the 25 years before and 25 years after with the Intcal20 interpolated curve. Take all of Intcal20 and place the actual data into the interpolated data. This would require rescaling the data with a linear curve or something along those lines. Only test the injection recovery at te real years. 

Linear combination of the basis vectors provided by the template, linear trend and constant.
$Ax = b$ simple linear regression with a 50 by 3 design matrix that is dotted with a 3 long position vector to produce the predicted d14c. The first row is just the template. The second is just rows and the final is just the year.

Resample the data with gaussian noise of the same magnitude as the error bars. Plot the amplitdues as a histogram. Calculate the chi squared statsitic and then do just trend and mean and calculate the difference in chi squared statistic. I choose the significance threshold of the resulting distribution.This gives the false positive rate.

Then you inject a number of small events and see if they are detected based on the thresholds you decided. This gives you the false negative rate. 

Repeating this for every year I can plot the year vs the 50% false positive amplitude of the event. Interpolate over all of the years